In [ ]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly

  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3900 sha256=6202cad9fae7de717a13fc023ee2a04ce58bd74ac7c2b5d4ddde149bb5a87703
  Stored in directory: c:\users\chris\appdata\local\pip\cache\wheels\85\5d\4e\7c276b57992951dbe770bf5caad6448d0539c510663aefd2e2
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4076 sha256=cddea4b5e3cb6027fdd9cca993c307b341f59af7d745a4bd7e9372ff8ce22335
  Stored in directory: c:\users\chris\appdata\local\pip\cache\wheels\ec\6b\81\05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3806 sha256=72b7cf566f9a96252ae66e18c2abe8a5ffb918a01903b445dde1e5717d836c47
  Stored in directory: c:\users\chris\appdata\local\pip\cache\wheels\00\f9\c7\1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
Successfully built dash-table dash-html-components dash-core-components


In [1]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
from dash.dependencies import Input, Output
from utils import get_modification_count_per_protein
import pandas as pd
import plotly.express as px

path = r"UHT milk P036.csv"
 
df = pd.read_csv(path)

def count_no_of_modifications(ptm_str):
    #check if NaN value
    if pd.isnull(ptm_str):
        return 0
    return 1 + ptm_str.count(';')

#apply count_no_of_modifications to each PTM column
df['#modifications'] = df['PTM'].apply(count_no_of_modifications)
#print non zero values in modifcations column
df[df['#modifications'] > 0]

app = dash.Dash()

colorscales = px.colors.named_colorscales()

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Distribution of modifications over different proteins', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
        dcc.Dropdown( id = 'norm_dropdown',
        style= {'padding': 5,},
        options = [
            {'label':'Normalization: nothing', "value":"" },
            {'label': 'Normalization: total protein modification count', 'value':"protein_total_mod_count"},
            {'label': 'Normalizaton: protein length', 'value':"protein_length"},
            {'label': 'Normalization: protein mass', 'value':"protein_mass"},
            {'label': 'Normalization: protein intensity', 'value':"protein_intensity"},
            ],
        value = ''),
        dcc.Dropdown( id = 'count_dropdown',
        style= {'padding': 5,},
        options = [
            {'label':'Only include proteins with modification count OVER 10', "value":10 },
            {'label': 'Only include proteins with modification count OVER 20', 'value':20},
            {'label': 'Only include proteins with modification count OVER 40', 'value':40},
            {'label': 'Only include proteins with modification count OVER 80', 'value':80},
            {'label': 'Only include proteins with modification count OVER 160', 'value':160},
            {'label': 'Only include proteins with modification count OVER 320', 'value':320},
            {'label': 'Only include proteins with modification count OVER 640', 'value':640},
            ],
        value = 80),
        dcc.Dropdown(
        id='colorscale', 
        style= {'padding': 5,},
        options=[{"value": x, "label": x} 
                 for x in colorscales],
        value='viridis'
        ),
        dcc.Graph(id = 'mod_heatmap_plot'),
                dcc.Loading(
            id="loading-1",
            type="default",
            children=html.Div(id="loading-output-1")
        ),
    ])
    
    
@app.callback(Output(component_id='mod_heatmap_plot', component_property= 'figure'),
              [Input(component_id='norm_dropdown', component_property= 'value'), Input(component_id='count_dropdown', component_property= 'value'), Input("colorscale", "value")])
def graph_update(dropdown_value_norm, dropdown_value_count, scale):
    print(dropdown_value_norm)
    print(dropdown_value_count)
    modPd = pd.DataFrame(get_modification_count_per_protein(df, dropdown_value_count, '{0}'.format(dropdown_value_norm)))
    fig = go.Figure(data=go.Heatmap(df_to_plotly(modPd), colorscale = scale))
    fig.update_layout(xaxis_title = 'Name of protein',
                      yaxis_title = 'Name of modification'
                      )
    return fig  


app.run_server()

C:\Users\chris\anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  
C:\Users\chris\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  after removing the cwd from sys.path.


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:32:55] "POST /_dash-update-component HTTP/1.1" 200 -



80
normalization: 
norm is: no normalization


127.0.0.1 - - [07/Oct/2021 10:33:03] "POST /_dash-update-component HTTP/1.1" 200 -


...

127.0.0.1 - - [07/Oct/2021 10:33:38] "POST /_dash-update-component HTTP/1.1" 200 -


protein_length
160
normalization: protein_length
norm is: protein_length


127.0.0.1 - - [07/Oct/2021 10:33:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 10:34:32] "POST /_dash-update-component HTTP/1.1" 200 -


protein_total_mod_count
160
normalization: protein_total_mod_count
norm is: protein_total_mod_count
P02666
P02662
P02668
P02754
B5B3R8
P00711
P02663
G9G9X6
protein_mass
160
normalization: protein_mass
norm is: protein_mass


127.0.0.1 - - [07/Oct/2021 10:40:42] "POST /_dash-update-component HTTP/1.1" 200 -
